# Libraries 📚

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import time
import seaborn as sns
import matplotlib.pyplot as plt

# Code 💻

## Loading saved matrices 

In [ ]:
# Load matrices from previously generated files
X_01 = np.load('matrices/X_01.npy')
y_01 = np.load('matrices/y_01.npy')
X_03 = np.load('matrices/X_03.npy')
y_03 = np.load('matrices/y_03.npy')
nX = np.load('matrices/nX.npy')
ny = np.load('matrices/ny.npy')

## Reviewing Shapes

In [ ]:
print(X_03.shape, y_03.shape)
print(X_01.shape, y_01.shape)
print(nX.shape, ny.shape)

In [ ]:
# Make the 0.1 the standard dataset
X = X_01
y = y_01

## Reshaping with np.pad with zeros for a 21x21 CNN aproach

In [ ]:
# Make the 0.1 the standard dataset
X = X_01
y = y_01

### Splitting the data

In [ ]:
# Splitting the data into training and testing sets
# We will use 80% of the data for training and 20% of the data for testing.
# We will use the function train_test_split from sklearn.model_selection to split the data.
# We will use the parameter stratify to split the data in a stratified way.
# We will split the training data into training and validation sets.
# We will use 80% of the training data for training and 20% of the training data for validation.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

In [ ]:
# Show target vector (1 = muon, 0 = antimuon)
print(y_train)

In [ ]:
random_events = np.random.randint(0, X_train.shape[0], 1)

for i in random_events:
        fig, axs = plt.subplots(1, 2, figsize=(12, 22))
                                        
        # Event
        axs[0].imshow(X_train[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if y_train[i] == 1 else 'Antimuon')
        axs[1].imshow(X_train[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if y_train[i] == 1 else 'Antimuon')

        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')

        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[1].set_xlabel('Side 2')
        plt.show()

### Padding

In [ ]:
# We want to bring the dimension from each matrix from 21,15,2 to 21,21,2.
# We will use numpy.pad to pad the matrices with zeros.
# We will pad the matrices with zeros in the third dimension (columns) and calculate the necessary padding for each matrix.

X_train = np.pad(X_train, ((0,0),(0,0),(0, X_train.shape[1]-X_train.shape[2]), (0,0)), mode='constant')
X_val = np.pad(X_val, ((0,0),(0,0),(0, X_val.shape[1]-X_val.shape[2]), (0,0)), mode='constant')
X_test = np.pad(X_test, ((0,0),(0,0),(0, X_test.shape[1]-X_test.shape[2]), (0,0)), mode='constant')

# Normalizing the data
# We will use the function normalize from keras.utils to normalize the data.
# We will normalize the data by dividing the data by the maximum value of the data.
X_train = keras.utils.normalize(X_train, axis=1)
X_val = keras.utils.normalize(X_val, axis=1)
X_test = keras.utils.normalize(X_test, axis=1)

# Reshape the y data to be a column one-hot encoded vector.
y_train = keras.utils.to_categorical(y_train, 2)
y_val = keras.utils.to_categorical(y_val, 2)
y_test = keras.utils.to_categorical(y_test, 2)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

In [ ]:
# Show target vector ([0,1] = muon, [1,0] = antimuon)
y_train

### Plot events

In [ ]:
for i in random_events:
        fig, axs = plt.subplots(1, 2, figsize=(12, 22))
                                        
        # Event
        axs[0].imshow(X_train[i,:,:,0], cmap='Reds')
        axs[0].set_title('Muon' if y_train[i][0] == 0 else 'Antimuon')
        axs[1].imshow(X_train[i,:,:,1], cmap='Blues')
        axs[1].set_title('Muon' if y_train[i][0] == 0 else 'Antimuon')

        # Show tick labels on both sides
        axs[0].tick_params('x', labelbottom=True, labeltop=True)
        axs[0].tick_params('y', labelleft=True, labelright=True)
        axs[0].set_xlabel('Side 1')

        axs[1].tick_params('x', labelbottom=True, labeltop=True)
        axs[1].tick_params('y', labelleft=True, labelright=True)
        axs[1].set_xlabel('Side 2')
        plt.show()
        

### CNN

In [ ]:
# CNN
# We will use a CNN to classify the events.
# The input of the CNN will be a matrix with two channels (side 1 and side 2) with the energy deposited in each sparse coordinate.
# The output of the CNN will be a vector with two elements (one for each class).
# The CNN will have two convolutional layers, two max pooling layers and two dense layers.
# The first convolutional layer will have 32 filters of size 3x3.
# The second convolutional layer will have 64 filters of size 3x3.
# The first max pooling layer will have a pool size of 2x2.
# The second max pooling layer will have a pool size of 2x2.
# The first dense layer will have 128 neurons.
# The second dense layer will have 2 neurons.

def f1_score(precision, recall):
        return 2 * (precision * recall) / (precision + recall)


# Create the CNN
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

# Compile the model and build F1 score metric
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.Precision(), keras.metrics.Recall(), 'accuracy', f1_score])

model.summary()

# Time the training
start = time.time()
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))
end = time.time()
print('Training time:', end - start)


### Plot

In [ ]:
# Plotting the training and validation loss, accuracy and F1 score
fig, axs = plt.subplots(1, 2, figsize=(20, 7))
axs[0].plot(history.history['loss'], label='Training loss')
axs[0].plot(history.history['val_loss'], label='Validation loss')
axs[0].set_title('Training and validation loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)
axs[1].plot(history.history['accuracy'], label='Training accuracy')
axs[1].plot(history.history['val_accuracy'], label='Validation accuracy')
axs[1].set_title('Training and validation accuracy')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend()
axs[1].grid(True)
plt.show()

In [ ]:
# Plotting the precision, recall and F1 score
fig, axs = plt.subplots(1, 3, figsize=(20, 7))
axs[0].plot(history.history['precision'], label='Training precision')
axs[0].plot(history.history['val_precision'], label='Validation precision')
axs[0].set_title('Training and validation precision')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Precision')
axs[0].legend()
axs[0].grid(True)
axs[1].plot(history.history['recall'], label='Training recall')
axs[1].plot(history.history['val_recall'], label='Validation recall')
axs[1].set_title('Training and validation recall')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Recall')
axs[1].legend()
axs[1].grid(True)
axs[2].plot(history.history['f1_score'], label='Training F1 score')
axs[2].plot(history.history['val_f1_score'], label='Validation F1 score')
axs[2].set_title('Training and validation F1 score')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('F1 score')
axs[2].legend()
axs[2].grid(True)
plt.show()

In [ ]:
# Obtain test loss and accuracy, recall, precision and F1 score
cnn_loss, cnn_precision, cnn_recall, cnn_accuracy, cnn_f1_score = model.evaluate(X_test, y_test)

In [ ]:
print(model.metrics_names)
print(cnn_loss, cnn_precision, cnn_recall, cnn_accuracy, cnn_f1_score)

In [ ]:
# Plot the confusion matrix
from sklearn.metrics import confusion_matrix

y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(model.predict(X_test), axis=1)

cm = confusion_matrix(y_true, y_pred)

def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(6, 6))
    plt.rcParams.update({'font.size': 20})
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()

class_names = ["Antimuons", "Muons"]  
plot_confusion_matrix(cm, classes=class_names)